# Scale

> Stack of intervals

In [ ]:
#|default_exp scale

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import numpy as np
import pandas as pd
from fastcore.all import *
from IPython.display import Audio
from itertools import combinations, accumulate
from collections import defaultdict

from musy import Note, Chord, Interval

In [ ]:
#|export
INTERVAL_NAMES = {
    "1": "unison",
    "b2": "minor second",
    "2": "major second",
    "#2": "augmented second",
    "b3": "minor third",
    "3": "major third",
    "#3": "augmented third",
    "b4": "minor fourth",
    "4": "perfect fourth",
    "#4": "augmented fourth",
    "b5": "diminished fifth",
    "5": "perfect fifth",
    "#5": "augmented fifth",
    "b6": "minor sixth",
    "6": "major sixth",
    "#6": "augmented sixth",
    "b7": "minor seventh",
    "7": "major seventh",
    "8": "octave",
    "b9": "minor ninth",
    "9": "major ninth",
    "#9": "augmented ninth",
    "b10": "minor tenth",
    "10": "major tenth",
    "b11": "minor eleventh",
    "11": "major eleventh",
    "#11": "augmented eleventh",
    "b12": "minor twelfth",
    "12": "major twelfth",
    "b13": "minor thirteenth",
    "13": "major thirteenth",
    "#13": "augmented thirteenth"
}
INTERVAL_HALF_STEPS = {
    "1": 0,
    "#1": 1,
    "b2": 1,
    "2": 2,
    "#2": 3,
    "b3": 3,
    "3": 4,
    "b4": 4,
    "4": 5,
    "#4": 6,
    "b5": 6,
    "5": 7,
    "#5": 8,
    "b6": 8,
    "6": 9,
    "#6": 10,
    "b7": 10,
    "7": 11,
    "8": 12,
    "b9": 13,
    "9": 14,
    "#9": 15,
    "10": 16,
    "11": 17,
    "#11": 18,
    "12": 19,
    "b13": 20,
    "13": 21,
    "#13": 22,
    "14": 23,
    "15": 24
}


SCALES = {# Major modes
          "ionian": ["1", "2", "3", "4", "5", "6", "7"],
          "major": ["1", "2", "3", "4", "5", "6", "7"],
          "minor": ["1", "2", "b3", "4", "5", "b6", "b7"],
          "natural minor": ["1", "2", "b3", "4", "5", "b6", "b7"],
          "dorian": ["1", "2", "b3", "4", "5", "6", "b7"],
          "phrygian": ["1", "b2", "b3", "4", "5", "b6", "b7"],
          "lydian": ["1", "2", "3", "#4", "5", "6", "7"],
          "mixolydian": ["1", "2", "3", "4", "5", "6", "b7"],
          "aeolian": ["1", "2", "b3", "4", "5", "b6", "b7"],
          "locrian": ["1", "b2", "b3", "4", "b5", "b6", "b7"],
          # Melodic Minor modes
          "melodic minor": ["1", "2", "b3", "4", "5", "6", "7"],
          "dorian b2": ["1", "b2", "b3", "4", "5", "6", "b7"],
          "lydian augmented": ["1", "2", "3", "#4", "#5", "6", "7"],
          "lydian b7": ["1", "2", "3", "#4", "5", "6", "b7"],
          "lydian dominant": ["1", "2", "3", "#4", "5", "6", "b7"],
          "acoustic": ["1", "2", "3", "#4", "5", "6", "b7"],
          "mixolydian #11": ["1", "2", "3", "#4", "5", "6", "b7"],
          "locrian n2": ["1", "2", "b3", "4", "b5", "b6", "7"],
          "altered": ["1", "b2", "b3", "b4", "b5", "b6", "b7"],
          "altered dominant": ["1", "b2", "b3", "b4", "b5", "b6", "b7"],
          "diminished whole-tone": ["1", "b2", "b3", "b4", "b5", "b6", "b7"],
          "dominant whole-tone": ["1", "b2", "b3", "b4", "b5", "b6", "b7"],
          "aeolian dominant": ["1", "2", "3", "4", "5", "b6", "b7"],
          # Harmonic minor modes
          "harmonic minor": ["1", "2", "b3", "4", "5", "b6", "7"],
          "locrian 6": ["1", "b2", "b3", "4", "b5", "6", "b7"],
          "ionian augmented": ["1", "2", "3", "4", "#5", "6", "7"],
          "dorian #4": ["1", "2", "b3", "#4", "5", "6", "b7"],
          "phrygian major": ["1", "b2", "3", "4", "5", "b6", "b7"],
          "phrygian dominant": ["1", "b2", "3", "4", "5", "b6", "b7"],
          "spanish phrygian": ["1", "b2", "3", "4", "5", "b6", "b7"],
          "spanish major": ["1", "b2", "3", "4", "5", "b6", "b7"],
          "spanish": ["1", "b2", "3", "4", "5", "b6", "b7"],
          "lydian #2": ["1", "#2", "3", "#4", "5", "6", "7"],
          "lydian #9": ["1", "#2", "3", "#4", "5", "6", "7"],
          "altered dominant bb7": ["1", "b2", "b3", "b4", "b5", "b6", "6"],
          "altered 13": ["1", "b2", "b3", "b4", "b5", "b6", "6"],
          # Harmonic major modes
          "harmonic major": ["1", "2", "3", "4", "5", "b6", "7"],
          "ionian b6": ["1", "2", "3", "4", "5", "b6", "7"],
          "dorian b5": ["1", "2", "b3", "4", "b5", "6", "b7"],
          "phrygian b4": ["1", "b2", "b3", "b4", "5", "b6", "b7"],
          "lydian b3": ["1", "2", "b3", "#4", "5", "6", "7"],
          "melodic minor #4": ["1", "2", "b3", "#4", "5", "6", "7"],
          "mixolydian b2": ["1", "b2", "3", "4", "5", "6", "b7"],
          # Lydian augmented already defined in melodic minor modes
          "locrian bb7": ["1", "b2", "b3", "4", "b5", "b6", "6"],
          # Double harmonic major modes
          "double harmonic major": ["1", "b2", "3", "4", "5", "b6", "7"],
          "double harmonic": ["1", "b2", "3", "4", "5", "b6", "7"],
          "lydian #2#6": ["1", "#2", "3", "#4", "5", "#6", "7"],
          "ultraphrygian": ["1", "b2", "b3", "b4", "5", "b6", "6"],
          "hungarian minor": ["1", "2", "b3", "#4", "5", "b6", "7"],
          "oriental": ["1", "b2", "3", "4", "b5", "6", "b7"], # Mixolydian b2 with lowered 5
          "ionian aug#2": ["1", "#2", "3", "4", "#5", "6", "7"],
          "locrian bb3bb7": ["1", "b2", "2", "4", "b5", "b6", "6"],
          # Bebop scales
          "bebop dominant": ["1", "2", "3", "4", "5", "6", "#6", "7"],
          "bebop major": ["1", "2", "3", "4", "5", "b6", "6", "7"],
          "bebop blues": ["1", "2", "b3", "3", "4", "5", "6", "b7"],
          "bebop melodic minor": ["1", "2", "b3", "4", "5", "b6", "6", "7"],
          # Pentatonic scales
          "pentatonic major": ["1", "2", "3", "5", "6"],
          "major pentatonic": ["1", "2", "3", "5", "6"],
          "pentatonic minor": ["1", "b3", "4", "5", "b7"],
          "minor pentatonic": ["1", "b3", "4", "5", "b7"],
          "pentatonic blues": ["1", "b3", "4", "b5", "5", "b7"],
          "blues pentatonic": ["1", "b3", "4", "b5", "5", "b7"],
          "pentatonic neutral": ["1", "2", "4", "5", "b7"],
          "neutral pentatonic": ["1", "2", "4", "5", "b7"],
          "pentatonic rock": ["1", "b3", "4", "#5", "b7"],
          "rock pentatonic": ["1", "b3", "4", "#5", "b7"],
          "jue": ["1", "b3", "4", "#5", "b7"], # Same intervals as rock pentatonic
          "pentatonic scottish": ["1", "2", "4", "5", "6"],
          "scottish pentatonic": ["1", "2", "4", "5", "6"],
          # Blues scales
          "blues major": ["1", "2", "b3", "3", "5", "6"],
          "blues minor": ["1", "b3", "4", "b5", "5", "b7"],
          "blues": ["1", "b3", "4", "b5", "5", "b7"], # Shorthand for Blues Minor
          "blues diminished": ["1", "b2", "b3", "3", "b5", "5", "6", "b7"],
          "aux diminished blues": ["1", "b2", "b3", "3", "b5", "5", "6", "b7"], 
          "half-whole": ["1", "b2", "b3", "3", "b5", "5", "6", "b7"],
          "balinese": ["1", "b2", "b3", "3", "b5", "5", "6", "b7"], 
          # Gypsy scales
          "gypsy major": ["1", "b2", "3", "4", "5", "b6", "7"],
          "gypsy minor": ["1", "2", "b3", "b5", "5", "b6", "7"],
          "spanish gypsy": ["1", "b2", "3", "4", "5", "b6", "b7"], # Same as Phrygian dominant
          "hungarian gypsy": ["1", "2", "b3", "#4", "5", "b6", "b7"], # Hungarian minor with lowered 7
          # Diminished scales
          "diminished": ["1", "2", "b3", "4", "b5", "b6", "6", "7"],
          "tonic diminished": ["1", "2", "b3", "4", "b5", "b6", "6", "7"],
          "whole-half": ["1", "2", "b3", "4", "#4", "#5", "6", "7"],
          "dominant diminished": ["1", "b2", "b3", "3", "#4", "5", "6", "b7"],
          "whole": ["1", "2", "3", "#4", "#5", "#6"],
          "whole-tone": ["1", "2", "3", "#4", "#5", "#6"],
          "aux augmented": ["1", "2", "3", "#4", "#5", "#6"],
          # Maqam
          "bayati shuri": ["1", "2", "b3", "4", "b5", "6", "b7"],
          "hijaz": ["1", "b2", "3", "4", "5", "b6", "b7"],
          "hijaz kar": ["1", "b2", "3", "4", "5", "6", "7"],
          "huzam": ["1", "b2", "b3", "3", "5", "b6", "b7"],
          "nikriz": ["1", "2", "3", "#4", "5", "6", "7"],
          "tunisian": ["1", "2", "3", "#4", "5", "6", "7"],
          "saba": ["1", "2", "b3", "3", "5", "b6", "b7"],
          "sabah": ["1", "2", "b3", "3", "5", "b6", "b7"],
          "suznak": ["1", "2", "3", "4", "5", "#5", "7"],
          "neveseri": ["1", "b2", "b3", "3", "5", "b6", "b7", "7"],
          # Greek
          "pireaus": ["1", "b2", "3", "#4", "5", "#5", "7"],
          "tsinganikos": ["1", "b2", "3", "4", "#4", "6", "#6"],
          # Hindustani
          "marwa": ["1", "b2", "3", "#4", "5", "6", "7"],
          "poorvi": ["1", "b2", "3", "#4", "5", "6", "7"],
          "segah": ["1", "#2", "3", "4", "5", "#5", "7"],
          "todi": ["1", "b2", "b3", "#4", "5", "b6", "7"],
          # Carnatic
          "charukeshi": ["1", "2", "3", "4", "5", "b6", "b7"],
          "dharmaavati": ["1", "2", "b3", "b5", "5", "6", "7"],
          "lataangi": ["1", "2", "3", "#4", "5", "6", "7"],
          "vachaspati": ["1", "2", "3", "#4", "5", "6", "#6"],
          "natakpriya": ["1", "b2", "b3", "4", "5", "6", "b7"],
          "rampriya": ["1", "b2", "3", "#4", "5", "6", "#6"],
          "suryakant": ["1", "b2", "3", "4", "5", "6", "7"],
          # Japanese
          "joshi akikaze": ["1", "2", "b3", "5", "6"],
          "akikaze joshi": ["1", "2", "b3", "5", "6"],
          "joshi hira": ["1", "2", "b3", "5", "b6"],
          "hira joshi": ["1", "2", "b3", "5", "b6"],
          "joshi iwato": ["1", "b2", "4", "b5", "b7"],
          "iwato joshi": ["1", "b2", "4", "b5", "b7"],
          "joshi kokin": ["1", "2", "4", "5", "b6"],
          "kokin joshi": ["1", "2", "4", "5", "b6"],
          "joshi kumoi": ["1", "b2", "4", "5", "b6"],
          "kumoi joshi": ["1", "b2", "4", "5", "b6"],
          "joshi okinawa": ["1", "3", "4", "5", "7"],
          "okinawa joshi": ["1", "3", "4", "5", "7"],
          "sen in": ["1", "b2", "4", "5", "b7"],
          "in sen": ["1", "b2", "4", "5", "b7"],
          # Misc.
          "chromatic": ["1", "b2", "2", "b3", "3", "4", "#4", "5", "#5", "6", "b7", "7"],
          "augmented": ["1", "#2", "3", "5", "#5", "7"],
          "enigmatic ascending": ["1", "b2", "3", "#4", "#5", "#6", "7"],
          "enigmatic descending": ["1", "b2", "3", "4", "b6", "b7", "7"],
          "hungarian major": ["1", "b3", "3", "b5", "5", "6", "b7"],
          "neapolitan major": ["1", "b2", "b3", "4", "5", "6", "7"],
          "neapolitan minor": ["1", "b2", "b3", "4", "5", "b6", "7"],
          "prometheus": ["1", "2", "3", "b5", "6", "b7"],
          "mystic": ["1", "2", "3", "b5", "6", "b7"],
          "prometheus neapolitan": ["1", "b2", "3", "b5", "6", "b7"], # Prometheus with lowered 2
          "spanish 8 tone": ["1", "b2", "b3", "3", "4", "b5", "b6", "b7"],
          }

# Invert scales so alternative names can be identified by intervals or name
d = defaultdict(list)
[d[tuple(v)].append(k) for k,v in SCALES.items()]
INV_SCALES_BY_INTERVAL = dict(d)
INV_SCALES_BY_NAME = {k: d[tuple(v)] for k,v in SCALES.items()}

In [ ]:
#|hide
assert len(SCALES) == len(INV_SCALES_BY_NAME)

The `Scale` object is a collection of intervals and agnostic to any key. From the basic scale, notes can be derived by providing a root note.

In [ ]:
#|export
class Scale:
    def __init__(self, name: str):
        self.name = name.lower()
        self.intervals = SCALES.get(self.name, [])
        self.flats = sum(1 for i in self.intervals if i.startswith("b"))
        self.sharps = sum(1 for i in self.intervals if i.startswith("#"))
        self.naturals = len(self.intervals) - self.flats - self.sharps

    @classmethod
    def available_scales(cls): return list(SCALES.keys())
    
    @classmethod
    def from_intervals(cls, intervals: list[str], name: str = None):
        """Create a custom scale from a list of intervals."""
        for i in intervals:
            assert i in list(INTERVAL_NAMES), f"Interval '{i}' not valid. Available intervals: '{list(INTERVAL_NAMES.keys())}'"
        # Infer name if not given
        custom_scale = cls(INV_SCALES_BY_INTERVAL.get(tuple(intervals), ["unknown"])[0] if not name else name)
        custom_scale.intervals = intervals
        return custom_scale
    
    @property
    def rel_semitones(self):
        return [INTERVAL_HALF_STEPS[interval] for interval in self.intervals]
    
    @property
    def abs_semitones(self):
        rel = self.rel_semitones
        abs = []
        for i, r in enumerate(rel[1:]):
            abs.append(r - rel[i])
        # Last remaining interval
        abs.append(12-sum(abs))
        return abs
    
    def __repr__(self): return f"Scale: {self.name.title()}. Intervals: {self.intervals}"
    def __eq__(self, other): return self.intervals == other.intervals
    def __ne__(self, other): return not self == other
    def __iter__(self) -> list[str]: return iter(self.intervals)
    def __len__(self): return len(self.intervals)

## Initialization

The `Scale` representation gives you the name and intervals of the scale.

In [ ]:
major = Scale("major")
major

Scale: Major. Intervals: ['1', '2', '3', '4', '5', '6', '7']

In [ ]:
dorian = Scale("dorian")
dorian

Scale: Dorian. Intervals: ['1', '2', 'b3', '4', '5', '6', 'b7']

In [ ]:
lydian = Scale("lydian")
lydian

Scale: Lydian. Intervals: ['1', '2', '3', '#4', '5', '6', '7']

### Properties

The `Scale` object stores how many flats, sharps and naturals are in the key.

For example, a major scale contains only naturals. No flats or sharps.

In [ ]:
major.naturals, major.sharps, major.flats

(7, 0, 0)

A dorian scale has 2 flats (the 3rd and 7th degrees).

In [ ]:
dorian.flats

2

A lydian scale has 1 sharp (the 4th degree).

In [ ]:
lydian.sharps

1

Listify the scale to get the intervals or call `.intervals`.

In [ ]:
list(lydian)

['1', '2', '3', '#4', '5', '6', '7']

Semitones for the invervals in a scale can be retrieved in an absolute (intervals between notes) or relative (intervals from root).

In [ ]:
assert major.rel_semitones == [0, 2, 4, 5, 7, 9, 11]
major.rel_semitones

[0, 2, 4, 5, 7, 9, 11]

In [ ]:
assert major.abs_semitones == [2, 2, 1, 2, 2, 2, 1]
assert sum(major.abs_semitones) == 12
major.abs_semitones

[2, 2, 1, 2, 2, 2, 1]

## Comparison

Scales can be compared to each other.

In [ ]:
assert Scale("minor") == Scale("aeolian") == Scale("natural minor")
assert not major == dorian
assert lydian != dorian
assert lydian == lydian
assert Scale("minor") == Scale("aeolian")

## Custom Scales

Almost all scales are available to initialize by name, but custom scales can be created with `from_intervals`. Consult `Scale.available_scales` to get the built-in scales.

For example, we can create a custom `persian` scale.

In [ ]:
persian = Scale.from_intervals(["1", "b2", "3", "4", "b5", "b6", "7"], "persian")
persian

Scale: Persian. Intervals: ['1', 'b2', '3', '4', 'b5', 'b6', '7']

If no name is given, `Scale` will try to infer the name.

In [ ]:
mel_minor = Scale.from_intervals(["1", "2", "b3", "4", "5", "6", "7"])
assert mel_minor.name == "melodic minor"
mel_minor

Scale: Melodic Minor. Intervals: ['1', '2', 'b3', '4', '5', '6', '7']

## Note Generation

From a `Scale` object, we can get the notes given a root note.

In [ ]:
#|export
@patch
def get_notes(self:Scale, root, oct=4):
    """Get the notes of a scale from a root note."""
    root = Note(root, oct=oct) if isinstance(root, str) else root
    return [root + int(INTERVAL_HALF_STEPS[i]) for i in self.intervals]

In [ ]:
c_major = Scale("major").get_notes("C")
assert [str(n) for n in c_major] == ['C', 'D', 'E', 'F', 'G', 'A', 'B']
c_major

[musy.note.Note(note='C', oct=4),
 musy.note.Note(note='D', oct=4),
 musy.note.Note(note='E', oct=4),
 musy.note.Note(note='F', oct=4),
 musy.note.Note(note='G', oct=4),
 musy.note.Note(note='A', oct=4),
 musy.note.Note(note='B', oct=4)]

In [ ]:
g_major = Scale("major").get_notes("G")
g_major

[musy.note.Note(note='G', oct=4),
 musy.note.Note(note='A', oct=4),
 musy.note.Note(note='B', oct=4),
 musy.note.Note(note='C', oct=5),
 musy.note.Note(note='D', oct=5),
 musy.note.Note(note='E', oct=5),
 musy.note.Note(note='F#', oct=5)]

In [ ]:
c_dorian = Scale("dorian").get_notes("C")
assert [str(n) for n in c_dorian] == ['C', 'D', 'D#', 'F', 'G', 'A', 'A#']
c_dorian

[musy.note.Note(note='C', oct=4),
 musy.note.Note(note='D', oct=4),
 musy.note.Note(note='D#', oct=4),
 musy.note.Note(note='F', oct=4),
 musy.note.Note(note='G', oct=4),
 musy.note.Note(note='A', oct=4),
 musy.note.Note(note='A#', oct=4)]

## Exhaustive Diatonic Chords

We can get all diatonic chords in the scale with `get_diatonic_chords`. 

In [ ]:
#|export
@patch
def get_diatonic_chords(self:Scale, root, min_notes=3):
    assert min_notes > 1, "min_notes must be greater than 1."
    notes = self.get_notes(root)
    return [Chord(combo) for n in range(min_notes, len(notes)+1) for combo in combinations(notes, n)]

For example, let's look at all the diatonic E chords in the C major scale we can identify.

In [ ]:
[c for c in Scale("major").get_diatonic_chords("C") if "E" in c.name()]

[Chord: 'E minor seventh, second inversion'. Notes: ['D4', 'E4', 'G4'],
 Chord: 'E minor seventh, second inversion'. Notes: ['D4', 'E4', 'B4'],
 Chord: 'E minor triad'. Notes: ['E4', 'G4', 'B4'],
 Chord: 'E suspended fourth triad'. Notes: ['E4', 'A4', 'B4'],
 Chord: 'Esus4|CM6'. Notes: ['C4', 'E4', 'A4', 'B4'],
 Chord: 'E suspended seventh, third inversion'. Notes: ['D4', 'E4', 'A4', 'B4']]

## Interval Names

You can easily get relative and absolute interval names. Both shorthand and full names are available.

In [ ]:
#|export
@patch
def rel_interval_names(self:Scale, short=False):
        return self.intervals if short else [INTERVAL_NAMES[i] for i in self.intervals[1:]]

@patch
def abs_interval_names(self:Scale, short=False):
    return [getattr(Interval.from_semitones(i), "long" if not short else "short") for i in self.abs_semitones]

In [ ]:
assert major.rel_interval_names() == ['major second', 'major third', 'perfect fourth', 'perfect fifth', 'major sixth', 'major seventh']
assert major.rel_interval_names(short=True) == ['1', '2', '3', '4', '5', '6', '7']
assert major.abs_interval_names() == ['major second', 'major second', 'minor second', 'major second', 'major second', 'major second', 'minor second']
assert major.abs_interval_names(short=True) == ['2', '2', 'b2', '2', '2', '2', 'b2']
major.rel_interval_names()

['major second',
 'major third',
 'perfect fourth',
 'perfect fifth',
 'major sixth',
 'major seventh']

In [ ]:
Scale("locrian").rel_interval_names()

['minor second',
 'minor third',
 'perfect fourth',
 'diminished fifth',
 'minor sixth',
 'minor seventh']

## Alternative Scale Names

Many scales have alternative names. We can derive them using `get_scale_names`.

In [ ]:
#|export
@patch
def get_scale_names(self:Scale):
    return INV_SCALES_BY_INTERVAL.get(tuple(self.intervals), [])

In [ ]:
assert major.get_scale_names() == ["ionian", "major"]
major.get_scale_names()

['ionian', 'major']

## Modes of scale

A mode is a scale derived from a note on the scale. We can build a new `Scale` on every step of a `Scale` by cycling through the modes.

For this to work, we first need to shift the semitones of our scale and get a new set of relative semitones.

In [ ]:
#|export
@patch
def _shift_abs_semitones(self:Scale, n):
    """ Shift the absolute semitones of a scale by n steps. """
    return self.abs_semitones[n:] + self.abs_semitones[:n]

@patch
def _shift_rel_semitones(self:Scale, n:int):
    """ Shift relative semitones by n steps. """
    return [0] + list(accumulate(self._shift_abs_semitones(n)[:-1]))

In [ ]:
assert major.abs_semitones == [2, 2, 1, 2, 2, 2, 1]
assert major._shift_abs_semitones(1) == [2, 1, 2, 2, 2, 1, 2]
# Semitones for dorian scale (shift semitones 1 to the left)
major._shift_abs_semitones(1)

[2, 1, 2, 2, 2, 1, 2]

When we accumulate the absolute semitones of a scale, we get the relative semitones.

In [ ]:
assert major.rel_semitones == [0, 2, 4, 5, 7, 9, 11]
assert major._shift_rel_semitones(1) == [0, 2, 3, 5, 7, 9, 10]
# Relative semitones for dorian scale
major._shift_rel_semitones(1)

[0, 2, 3, 5, 7, 9, 10]

We can now reconstruct the intervals from the relative semitones.

In [ ]:
#|export
def semi_to_intvals(semitones):
    """Convert relative semitone values to interval names."""
    # Major scale is the baseline for accidentals
    maj = [0, 2, 4, 5, 7, 9, 11, 12, 14, 16, 17, 19, 21, 23, 24, 26, 28, 29, 31, 33, 35]
    acc = {-1: 'b', 0: '', 1: '#'}
    intervals = []
    for i, s in enumerate(semitones):
        deg = i + 1
        diff = (s - maj[i]) % 12
        if diff > 6: diff -= 12
        if diff == 2: intervals.append(f'{deg+1}')
        elif diff == -2: intervals.append(f'{deg-1}')
        else: intervals.append(f'{acc.get(diff, "")}{deg}')
    return intervals

In [ ]:
dorian_intervals = semi_to_intvals(major._shift_rel_semitones(1))
assert dorian_intervals == ['1', '2', 'b3', '4', '5', '6', 'b7']
dorian_intervals

['1', '2', 'b3', '4', '5', '6', 'b7']

Now let's reconstruct a new `Scale` object from these intervals.

In [ ]:
dorian = Scale.from_intervals(dorian_intervals)
assert dorian.name == "dorian"
dorian

Scale: Dorian. Intervals: ['1', '2', 'b3', '4', '5', '6', 'b7']

Now we can create a method to retrieve all modes for any given `Scale`.

In [ ]:
#|export
@patch
def shift_intvals(self:Scale, n:int) -> list[str]:
    """Shift the intervals of a scale by n steps."""
    return semi_to_intvals(self._shift_rel_semitones(n))

@patch
def get_modes(self:Scale) -> list[Scale]:
    return [Scale.from_intervals(self.shift_intvals(i)) for i in range_of(self)]

The most well known modes are the modes of the major scale, which are:
1. `Ionian` (Major)
2. `Dorian`
3. `Phrygian`
4. `Lydian`
5. `Mixolydian`
6. `Aeolian` (Natural Minor)
7. `Locrian`






In [ ]:
maj_modes = major.get_modes()
assert [m.name for m in maj_modes] == ["ionian", "dorian", "phrygian", "lydian", "mixolydian", "minor", "locrian"]
maj_modes

[Scale: Ionian. Intervals: ['1', '2', '3', '4', '5', '6', '7'],
 Scale: Dorian. Intervals: ['1', '2', 'b3', '4', '5', '6', 'b7'],
 Scale: Phrygian. Intervals: ['1', 'b2', 'b3', '4', '5', 'b6', 'b7'],
 Scale: Lydian. Intervals: ['1', '2', '3', '#4', '5', '6', '7'],
 Scale: Mixolydian. Intervals: ['1', '2', '3', '4', '5', '6', 'b7'],
 Scale: Minor. Intervals: ['1', '2', 'b3', '4', '5', 'b6', 'b7'],
 Scale: Locrian. Intervals: ['1', 'b2', 'b3', '4', 'b5', 'b6', 'b7']]

If we take the modes of one of these modes we get the same modes in a different order. For example, the phrygian modes are:
1. `Phrygian`
2. `Lydian`
3. `Mixolydian`
4. `Aeolian` (Natural Minor)
5. `Locrian`
6. `Ionian` (Major)






In [ ]:
phrygian = maj_modes[2]
phrygian_modes = phrygian.get_modes()
assert [m.name for m in phrygian_modes] == ["phrygian", "lydian", "mixolydian", "minor", "locrian", "ionian", "dorian"]
phrygian_modes

[Scale: Phrygian. Intervals: ['1', 'b2', 'b3', '4', '5', 'b6', 'b7'],
 Scale: Lydian. Intervals: ['1', '2', '3', '#4', '5', '6', '7'],
 Scale: Mixolydian. Intervals: ['1', '2', '3', '4', '5', '6', 'b7'],
 Scale: Minor. Intervals: ['1', '2', 'b3', '4', '5', 'b6', 'b7'],
 Scale: Locrian. Intervals: ['1', 'b2', 'b3', '4', 'b5', 'b6', 'b7'],
 Scale: Ionian. Intervals: ['1', '2', '3', '4', '5', '6', '7'],
 Scale: Dorian. Intervals: ['1', '2', 'b3', '4', '5', '6', 'b7']]

Let's take a more complicated example, like the `Double Harmonic Major` scale. The modes of this scale are:
1. Double Harmonic Major
2. Lydian with `#2` and `#6`
3. Ultraphrygian
4. Hungarian Minor
5. Oriental
6. Ionian with `#2` and `#5` (i.e. `aug#2`)
7. Locrian with `bb3` and `bb7`

In [ ]:
dhm = Scale("double harmonic major")
dhm

Scale: Double Harmonic Major. Intervals: ['1', 'b2', '3', '4', '5', 'b6', '7']

In [ ]:
dhm_modes = dhm.get_modes()
assert [m.name for m in dhm_modes] == ["double harmonic major", "lydian #2#6", "ultraphrygian", "hungarian minor", "oriental", "ionian aug#2", "locrian bb3bb7"]
dhm_modes

[Scale: Double Harmonic Major. Intervals: ['1', 'b2', '3', '4', '5', 'b6', '7'],
 Scale: Lydian #2#6. Intervals: ['1', '#2', '3', '#4', '5', '#6', '7'],
 Scale: Ultraphrygian. Intervals: ['1', 'b2', 'b3', 'b4', '5', 'b6', '6'],
 Scale: Hungarian Minor. Intervals: ['1', '2', 'b3', '#4', '5', 'b6', '7'],
 Scale: Oriental. Intervals: ['1', 'b2', '3', '4', 'b5', '6', 'b7'],
 Scale: Ionian Aug#2. Intervals: ['1', '#2', '3', '4', '#5', '6', '7'],
 Scale: Locrian Bb3Bb7. Intervals: ['1', 'b2', '2', '4', 'b5', 'b6', '6']]

## Scale Audio

Like with `Note` and `Chord` objects, `Scale` objects can be played in any given key with the `play` method.

In [ ]:
#|export
@patch
def get_audio_array(self:Scale, root, oct=4, length=0.3):
    notes = self.get_notes(root, oct=oct)
    octave = Note(root, oct=oct+1).get_audio_array(length=length)
    return np.concatenate([n.get_audio_array(length) for n in notes] + [octave])

@patch
def play(self:Scale, root, oct=4, length=0.3): 
    return Audio(self.get_audio_array(root, oct=oct, length=length), rate=44100)

In [ ]:
major.play("C")

In [ ]:
persian.play("C")

`length` and `oct` can be customized for the `play` method.

In [ ]:
Scale("altered").play("C", length=0.2, oct=3)

In [ ]:
Scale("aeolian dominant").play("C", length=0.2, oct=3)

## Triads

We can derive all triads in the `Scale`.

### Derive Triads

In [ ]:
#|export
@patch
def get_triads(self:Scale, root):
    """Get all triads in scale starting from root note."""
    notes = self.get_notes(root)
    return [Chord([notes[i], 
                  Note(str(notes[(i+2)%7]), oct=notes[i].oct + (i+2)//7),
                  Note(str(notes[(i+4)%7]), oct=notes[i].oct + (i+4)//7)]) 
            for i in range(len(notes))]

In [ ]:
major.get_triads("C")

[Chord: 'C major triad'. Notes: ['C4', 'E4', 'G4'],
 Chord: 'D minor triad'. Notes: ['D4', 'F4', 'A4'],
 Chord: 'E minor triad'. Notes: ['E4', 'G4', 'B4'],
 Chord: 'F major triad'. Notes: ['F4', 'A4', 'C5'],
 Chord: 'G major triad'. Notes: ['G4', 'B4', 'D5'],
 Chord: 'A minor triad'. Notes: ['A4', 'C5', 'E5'],
 Chord: 'B diminished triad'. Notes: ['B4', 'D5', 'F5']]

In [ ]:
Scale("phrygian").get_triads("D")

[Chord: 'D minor triad'. Notes: ['D4', 'F4', 'A4'],
 Chord: 'No chord found.'. Notes: ['D#4', 'G4', 'A#4'],
 Chord: 'F major triad'. Notes: ['F4', 'A4', 'C4'],
 Chord: 'No chord found.'. Notes: ['G4', 'A#4', 'D5'],
 Chord: 'No chord found.'. Notes: ['A4', 'C4', 'D#5'],
 Chord: 'No chord found.'. Notes: ['A#4', 'D5', 'F5'],
 Chord: 'No chord found.'. Notes: ['C5', 'D#6', 'G6']]

### Audio Triads

All triads in a given `Scale` can be played.

In [ ]:
#|export
@patch
def play_triads(self:Scale, root):
    """Play all triads in scale starting from root note."""
    return Audio(np.concatenate([c.get_audio_array() for c in self.get_triads(root)]), rate=44100)

In [ ]:
major.play_triads("C")

## Scale Seventh Chords

### Deriving Seventh Chords

We can also get all seventh chords in the `Scale`.

In [ ]:
#|export
@patch
def get_sevenths(self:Scale, root):
    """Get all seventh chords in scale starting from root note."""
    notes = self.get_notes(root)
    return [Chord([notes[i], 
                  Note(str(notes[(i+2)%7]), oct=notes[i].oct + (i+2)//7),
                  Note(str(notes[(i+4)%7]), oct=notes[i].oct + (i+4)//7),
                  Note(str(notes[(i+6)%7]), oct=notes[i].oct + (i+6)//7)]) 
            for i in range(len(notes))]

In [ ]:
c_seventh_triads = major.get_sevenths("C")
c_seventh_triads

[Chord: 'C major seventh'. Notes: ['C4', 'E4', 'G4', 'B4'],
 Chord: 'D minor seventh'. Notes: ['D4', 'F4', 'A4', 'C5'],
 Chord: 'E minor seventh'. Notes: ['E4', 'G4', 'B4', 'D5'],
 Chord: 'F major seventh'. Notes: ['F4', 'A4', 'C5', 'E5'],
 Chord: 'G dominant seventh'. Notes: ['G4', 'B4', 'D5', 'F5'],
 Chord: 'A minor seventh'. Notes: ['A4', 'C5', 'E5', 'G5'],
 Chord: 'B half diminished seventh'. Notes: ['B4', 'D5', 'F5', 'A5']]

### Audio Seventh Chords

In [ ]:
#|export
@patch
def play_sevenths(self:Scale, root):
    """Play all seventh chords in scale starting from root note."""
    return Audio(np.concatenate([c.get_audio_array() for c in self.get_sevenths(root)]), rate=44100)

In [ ]:
major.play_sevenths("C")

In [ ]:
lydian.get_sevenths("D")

[Chord: 'D major seventh'. Notes: ['D4', 'F#4', 'A4', 'C#4'],
 Chord: 'E dominant seventh'. Notes: ['E4', 'G#4', 'B4', 'D5'],
 Chord: 'F# minor seventh'. Notes: ['F#4', 'A4', 'C#4', 'E5'],
 Chord: 'G# half diminished seventh'. Notes: ['G#4', 'B4', 'D5', 'F#5'],
 Chord: 'A major seventh'. Notes: ['A4', 'C#4', 'E5', 'G#5'],
 Chord: 'B minor seventh'. Notes: ['B4', 'D5', 'F#5', 'A5'],
 Chord: 'C# minor seventh'. Notes: ['C#5', 'E6', 'G#6', 'B6']]

## Scale Table

We can display all relevant information from the scale in a Pandas DataFrame table.

In [ ]:
#|export
@patch
def to_frame(self:Scale, root=None):
    d = {
        "Degree": self.intervals,
        "Relative Interval": ["unison"] + self.rel_interval_names(),
        "Mode": [m.name for m in self.get_modes()],
        "Relative Semitones": self.rel_semitones,
        "Absolute Semitones": self.abs_semitones,
    }
    if root:
        d.update({
            "Notes": self.get_notes(root),
            "Triad": [t.name() for t in self.get_triads(root)],
            "Seventh Chord": [s.name() for s in self.get_sevenths(root)],
        })
    return pd.DataFrame(d)

The `to_frame` method can be called without a root note, but is most informative when given a root note. In that case it shows additional information such as the triads and seventh chords.


In [ ]:
major.to_frame(root="D")

,Degree,Relative Interval,Mode,Relative Semitones,Absolute Semitones,Notes,Triad,Seventh Chord
0,1,unison,ionian,0,2,D,D major triad,D major seventh
1,2,major second,dorian,2,2,E,E minor triad,E minor seventh
2,3,major third,phrygian,4,1,F#,F# minor triad,F# minor seventh
3,4,perfect fourth,lydian,5,2,G,G major triad,G major seventh
4,5,perfect fifth,mixolydian,7,2,A,A major triad,A dominant seventh
5,6,major sixth,minor,9,2,B,B minor triad,B minor seventh
6,7,major seventh,locrian,11,1,C#,C# diminished triad,C# half diminished seventh


TODO: Comparison method for scales

-------------------

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()